In [ ]:
#| echo : False
%load_ext autoreload
%autoreload 2
import sys
from fastcore.xtras import Path
sys.path.insert(0, Path.cwd().parent)

# Anatel - Consulta e Processamento do Banco de Dados

> Este repositório concentra um conjunto de scripts para navegar e baixar informações dos principais bancos de dados da Anatel e as bases públicas da Aeronáutica como o AISWEB, GEOAISWEB e ICAO cruzadas com documentações técnicas internas. Cujo dados serão utilizados em tarefas fiscalizatórias. O público alvo são os servidores do órgão, uma vez que a maioria dos sistemas utilizados aqui necessitam de autenticação cujo acesso é restrito aos servidores da ANATEL.

<a href="https://gitmoji.dev">
  <img src="https://img.shields.io/badge/gitmoji-%20😜%20😍-FFDD67.svg?style=flat-square" alt="Gitmoji">
</a>


[![scraper: gazpacho](https://img.shields.io/badge/scraper-gazpacho-C6422C)](https://github.com/maxhumber/gazpacho)

## Instalação

<code>pip install anateldb</code>

## Como utilizar

### Consulta à base de dados formatada para o AppAnalise de Anatel

A motivação original para a presente biblioteca foi disponibilizar os dados de diferentes fontes da Anatel - e posteriormente da Aeronáutica - programaticamente, para serem utilizadas pelo [AppAnalise](https://github.com/EricMagalhaesDelgado/appAnalise/releases/) na identificação de emissões captadas pelos planos de monitoração e também em fiscalização de campo.

Os dados no formato atual em excel são disponibilizados na pasta dados no repositório https://github.com/ronaldokun/anateldb. 

Os dados em formato otimizado `.parquet.gzip` são disponibilizados junto ao repositório na pasta `dados` juntamente com as versões das bases individuais. Futuramente o objetivo é descontinuar a liberação de arquivos em formato excel e disponibilizar apenas os dados em formato otimizado. 

In [ ]:
from fastcore.xtras import Path
from extracao.main import get_db
from nbdev.showdoc import *

pasta = Path.cwd().parent / 'dados'
show_doc(get_db)

---

#### get_db

>      get_db (path:Union[str,pathlib.Path], update:bool=False, dist:float=10)

Lê e opcionalmente atualiza as bases da Anatel, mescla as bases da Aeronáutica, salva e retorna o arquivo

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| path | typing.Union[str, pathlib.Path] |  | Pasta onde salvar os arquivos", |
| update | bool | False | Atualizar as bases da Anatel e da Aeronáutica?", |
| dist | float | 10 | Distância máxima entre as coordenadas consideradas iguais |
| **Returns** | **DataFrame** |  | **Retorna o DataFrame com as bases da Anatel e da Aeronáutica** |

In [ ]:
db = get_db(pasta)
db.sample(5)

📜Lendo as bases de dados da Anatel...

,Id,Frequency,Latitude,Longitude,Description,Service,Station,Class,BW
667035,#667036,527.00000,-2.806110,-57.069721,"[MOS] TV-C1, C, Camara Dos Deputados (50441490...",801,-1,NI,5700.0
107525,#107526,156.47500,-8.581717,-63.725052,"[STEL] L, ML, Transportes Bertolini Ltda (1202...",19,1000751993,G3E,16.0
117314,#117315,156.70000,-1.540750,-48.743168,"[STEL] L, FC, Alpina Briggs Defesa Ambiental S...",19,690246978,F3E,16.0
273145,#273146,167.63125,-20.387291,-44.484554,"[STEL] L, ML, Policia Militar Do Estado De Min...",19,1001948332,F1W,8.1
306394,#306395,167.74375,-19.881208,-43.859077,"[STEL] L, ML, Policia Militar Do Estado De Min...",19,1002234406,G1W,8.1


### Métodos para baixar ou atualizar os arquivos das bases de dados

```python
from anateldb.updates import update_mosaico, update_radcom, update_stel, update_base
```

A função seguinte baixa os dados diretamente da interface pública online do [Spectrum E](http://sistemas.anatel.gov.br/se/public/view/b/srd.php) 

```python
%%time
update_mosaico(pasta='D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```

    Baixando as Estações do Mosaico...
    Baixando o Plano Básico das Estações...
    Baixando o Histórico de Atualizações...
    Kbô
    Wall time: 8.12 s


```python
%%time
update_radcom('D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```

    Lendo o Banco de Dados de Radcom
    Wall time: 1 s

A função <code>update_stel</code> é bem mais lenta que as demais, dado que o banco de dados do STEL é antigo e abarca todos os registros de outorga de serviços de telecomunicações da ANATEL, com mais de **400.000** registros ativos. Esse banco de dados é atualizado 1 vez ao dia à meia-noite e remete ao estado do dia anterior, portanto não faz sentido atualizá-lo mais de 1 vez por dia.

```python
%%time
update_stel('D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```


A função `update_base` combina as 3 bases anteriores e uniformiza os campos:
```python
update_base('D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```


### Métodos para ler as Bases de Dados

In [ ]:
from extracao.reading import read_radcom, read_stel, read_mosaico, read_base
radcom = read_radcom(pasta)
radcom.sample(5).iloc[:,:5]

,Frequência,Entidade,Fistel,Número_Estação,Município
1681,87.9,ASSOCIACAO DOS MORADORES DO SANA,50410717991,1001069185,Macaé
623,87.9,ASSOC.COM.DE VOL.P/ EVENTOS BENEFICENTES DE ST...,50406291411,692621105,Santa Fé de Goiás
1451,87.9,ASSOCIACAO E MOVIMENTO COMUNITARIO DE JATAUBA,50403889006,689546351,Jataúba
4750,106.3,ASSOCIACAO CULTURAL DE GUIMARAES,50013418840,683546821,Guimarães
885,87.9,COMUNIDADE FAMENSE DE RADIO,50407180125,695539701,Fama


Se o argumento <code>update=True</code> for fornecido ou arquivo local não existir, a base de dados é atualizada por meio da função <code>update_radcom</code>. 

> *A função <code>update_radcom</code> somente irá funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*

In [ ]:
stel = read_stel(pasta)
stel.sample(5).iloc[:,:7]

,Frequência,Classe_Emissão,Largura_Emissão,Classe,Num_Serviço,Entidade,Fistel
713502,459.68750,F1D,7K60,FX,019,MOSAIC FERTILIZANTES P&K LTDA.,50416997473
744967,469.70000,F1W,7K60,BR,019,ASSOCIACAO MELHORAMENTOS PARQUE SILVINO PEREIRA,50011329165
353408,167.34375,F1W,7K60,FB,019,DURATEX FLORESTAL LTDA,02021684504
226524,157.78125,F1E,7K60,FX,011,"OLIVEIRA, TAVORA E CIA LTDA",50418317186
380431,167.68125,F3E,11K0,ML,019,POLICIA MILITAR DO ESTADO DE MINAS GERAIS,50401288943


Se o argumento <code>update=True</code> for fornecido ou arquivo local não existir, a base de dados é atualizada por meio da função <code>update_stel</code>. 

> *A função  <code>update_stel</code> somente irá funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*

In [ ]:
mosaico = read_mosaico(pasta)
mosaico.sample(5).iloc[:,:7]

,Num_Serviço,Id,Número_Estação,Latitude,Longitude,Validade_RF,Município
14017,800,57dbabf18de3d,1002434758,-21.762528,-43.357556,2023-12-31,Juiz de Fora
28419,801,617afd492ade4,<NA>,-17.400000,-42.717220,2041-11-18,Veredinha
216,800,57dbaadbc8413,441611648,-7.730833,-72.648889,2023-12-31,Rodrigues Alves
15976,230,57dbac17c5a24,699381339,-19.817611,-40.336056,2024-08-20,Ibiraçu
10352,801,57dbabb50b92a,<NA>,-21.208611,-45.198056,2035-10-09,Nepomuceno


Se o argumento <code>update=True</code> for fornecido ou o arquivo local não existir, a base de dados é atualizada por meio da função <code>update_mosaico</code>. 

> A função <code>update_mosaico</code> usa a base de dados Pública do Spectrum E, portanto basta somente estar conectado na internet &#x1F60E;.

A função `update_base` combina as 3 atualizações de base anteriormente descritas.